In [1]:
import os
import threading

import seaborn as sns

from causal_models.confounder_mediator_scms import LinearConfounderMediatorSCM
from frontdoor_backdoor_main import execute_strategy_iteration
from utils import parallel_utils, plot_utils

In [2]:
REPO_DIR = "/path/to/repo/directory"
LOG_DIR = os.path.join(REPO_DIR, "logs", "fd_bd")
LOG_PATH = os.path.join(LOG_DIR, "progress.txt")
RESULTS_DIR = os.path.join(REPO_DIR, "results")
PLOTS_DIR = os.path.join(RESULTS_DIR, "plots")

In [2]:
true_scm = LinearConfounderMediatorSCM(
    a=1, b=0.1, d=1, w=1, x=0, m0=0.3, m1=0.6, y=0.1
)

In [5]:
def run_in_background(results_dict):
    parallel_utils.get_timeseries_for(
        true_scm,
        [
            "oracle_with_true_nu",
            "fixed_equal",
            "etc_0.1",
            "etg_0.1",
            "etc_0.2",
            "etg_0.2",
            "etc_0.4",
            "etg_0.4"
        ],
        horizons=[2000, 4000, 6000, 8000],
        iterations=16000,
        results_dict=results_dict,
        execute_fn=execute_strategy_iteration,
        log_path=LOG_PATH,
    )

In [1]:
results_dict = {}
thread = threading.Thread(target=run_in_background, args=(results_dict,))
thread.start()

In [1]:
thread.is_alive()

In [6]:
palette = sns.color_palette("colorblind")
name_to_linestyle_color = {
    "etg_0.1": ["dotted", palette[2], "^"],
    "etg_0.2": ["loosely dashed", palette[3], "D"],
    "etg_0.4": ["densely dashed", palette[4], "3"],
    "fixed_equal": ["dashdotted", palette[1], "v"],
}

name_to_label = {"fixed_equal": "fixed"}

In [2]:
plot_utils.plot_regret_curve(
    results_dict=results_dict,
    name_to_linestyle_color=name_to_linestyle_color,
    name_to_label=name_to_label,
)